[참고] https://greendreamtrre.tistory.com/268

In [1]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

# Url
xmlUrl = 'http://apis.data.go.kr/6300000/parkInfoDaejeonService/parkInfoDaejeonList'

#service Key
My_API_Key = unquote('6Eh6zlwHnat49SS5qno4IFr8BxLGMYPpRPpRSPXx9NG+FDMhOaKrZ2pr8dAhgnbbuMaBaPSme/6qDiw4g9bORQ==')
queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : My_API_Key,    # 필수 항목 1 : 인증키
        quote_plus('pageNo') : '1',         #페이지 넘버
        quote_plus('numOfRows') : '1000'        #한 페이지에 몇개를 가져올건지
    }
)

#get, encode
response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

In [2]:
xmlobj

<?xml version="1.0" encoding="utf-8"?>
<ServiceResult><comMsgHeader><returnCode>00</returnCode><returnMessage>NORMAL_SERVICE</returnMessage><successYN>Y</successYN></comMsgHeader><msgHeader><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>412</totalCount><totalPage>1</totalPage></msgHeader><MsgBody><items><address>대전광역시 대덕구 장동 457 일원</address><latitude>36.406849</latitude><longitude>127.441681</longitude><manageInstitution>대전광역시 공원관리사업소(공원관리과)</manageInstitution><managementNumber>63000-00016</managementNumber><ntatcSeq>1081734097</ntatcSeq><parkArea>85488</parkArea><regDtTm>2015-12-31</regDtTm><section>문화공원</section><tel>042-270-7860</tel><title>장동</title><zipcode>우편번호는 제공되지 않습니다</zipcode></items><items><address>대전광역시 대덕구 연축동 산31 일원</address><latitude>36.388015</latitude><longitude>127.437176</longitude><manageInstitution>대전광역시 공원관리사업소(공원관리과)</manageInstitution><managementNumber>63000-00015</managementNumber><ntatcSeq>1081734096</ntatcSeq><parkArea>264600</parkArea><regDtTm>2015-

In [3]:
rows = xmlobj.findAll('items')

In [4]:
rows[0]

<items><address>대전광역시 대덕구 장동 457 일원</address><latitude>36.406849</latitude><longitude>127.441681</longitude><manageInstitution>대전광역시 공원관리사업소(공원관리과)</manageInstitution><managementNumber>63000-00016</managementNumber><ntatcSeq>1081734097</ntatcSeq><parkArea>85488</parkArea><regDtTm>2015-12-31</regDtTm><section>문화공원</section><tel>042-270-7860</tel><title>장동</title><zipcode>우편번호는 제공되지 않습니다</zipcode></items>

In [5]:
#리스트에 담기
columns = rows[0].find_all()

In [6]:
columns

[<address>대전광역시 대덕구 장동 457 일원</address>,
 <latitude>36.406849</latitude>,
 <longitude>127.441681</longitude>,
 <manageInstitution>대전광역시 공원관리사업소(공원관리과)</manageInstitution>,
 <managementNumber>63000-00016</managementNumber>,
 <ntatcSeq>1081734097</ntatcSeq>,
 <parkArea>85488</parkArea>,
 <regDtTm>2015-12-31</regDtTm>,
 <section>문화공원</section>,
 <tel>042-270-7860</tel>,
 <title>장동</title>,
 <zipcode>우편번호는 제공되지 않습니다</zipcode>]

In [7]:
# 행, 열 매트리스로 만들기
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없음)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행을 저장해야함    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워줌 
    
result = pd.DataFrame(rowList, columns=nameList)
result.head()

,address,latitude,longitude,manageInstitution,managementNumber,ntatcSeq,parkArea,regDtTm,section,tel,title,zipcode
0,대전광역시 대덕구 장동 457 일원,36.406849,127.441681,대전광역시 공원관리사업소(공원관리과),63000-00016,1081734097,85488,2015-12-31,문화공원,042-270-7860,장동,우편번호는 제공되지 않습니다
1,대전광역시 대덕구 연축동 산31 일원,36.388015,127.437176,대전광역시 공원관리사업소(공원관리과),63000-00015,1081734096,264600,2015-12-31,근린공원,042-270-7860,연축,우편번호는 제공되지 않습니다
2,대전광역시 동구 효평동 산84-1 일원,36.393979,127.455059,대전광역시 공원관리사업소(공원관리과),63000-00014,1081734095,82032,2015-12-31,역사공원,042-270-7860,계족산성,우편번호는 제공되지 않습니다
3,대전광역시 대덕구 장동 130-2 일원,36.403092,127.445508,대전광역시 공원관리사업소(공원관리과),63000-00013,1081734094,86800,2015-12-31,문화공원,042-270-7860,산디,우편번호는 제공되지 않습니다
4,대전광역시 대덕구 읍내동 산1-9 일원,36.381933,127.437533,대전광역시 공원관리사업소(공원관리과),63000-00012,1081734093,347600,2015-12-31,문화공원,042-270-7860,읍내,우편번호는 제공되지 않습니다


In [8]:
#data shape
result.shape

(412, 12)

In [9]:
#우편번호는 제공하지 않기때문에 삭제 
result = result.drop('zipcode', axis=1)
result

,address,latitude,longitude,manageInstitution,managementNumber,ntatcSeq,parkArea,regDtTm,section,tel,title
0,대전광역시 대덕구 장동 457 일원,36.406849,127.441681,대전광역시 공원관리사업소(공원관리과),63000-00016,1081734097,85488,2015-12-31,문화공원,042-270-7860,장동
1,대전광역시 대덕구 연축동 산31 일원,36.388015,127.437176,대전광역시 공원관리사업소(공원관리과),63000-00015,1081734096,264600,2015-12-31,근린공원,042-270-7860,연축
2,대전광역시 동구 효평동 산84-1 일원,36.393979,127.455059,대전광역시 공원관리사업소(공원관리과),63000-00014,1081734095,82032,2015-12-31,역사공원,042-270-7860,계족산성
3,대전광역시 대덕구 장동 130-2 일원,36.403092,127.445508,대전광역시 공원관리사업소(공원관리과),63000-00013,1081734094,86800,2015-12-31,문화공원,042-270-7860,산디
4,대전광역시 대덕구 읍내동 산1-9 일원,36.381933,127.437533,대전광역시 공원관리사업소(공원관리과),63000-00012,1081734093,347600,2015-12-31,문화공원,042-270-7860,읍내
...,...,...,...,...,...,...,...,...,...,...,...
407,대전광역시 중구 호동 산101번지 일원,36.2975369,127.437172799999,대전광역시 공원관리사업소(공원관리과),30000-00005,1051456,407900,2016-06-10,근린공원,042-270-7860,호동근린공원
408,대전광역시 중구 대사동 190-11번지,36.3094513,127.4239611,대전광역시 공원관리사업소(공원관리과),30000-00004,1051455,757000,2016-06-10,근린공원,042-270-7860,대사근린공원
409,대전광역시 중구 사정동 산2번지,36.3007952,127.407088599999,대전광역시 공원관리사업소(공원관리과),30000-00003,1051454,1288000,2016-06-10,근린공원,042-270-7860,사정근린공원
410,대전광역시 서구 둔산대로 169,36.215831,127.231643,대전광역시 한밭수목원,30000-00002,1051453,569340.6,2016-06-10,근린공원,042-270-8452,둔산대공원


In [10]:
result['address'][0][6:9]

'대덕구'

In [11]:
dong = []
for i in range(result.shape[0]):
    dong_3 = result.address[i][6:9]
    dong.append(dong_3.replace(" ",""))
dong[:5]

['대덕구', '대덕구', '동구', '대덕구', '대덕구']

In [12]:
result['dong'] = dong

In [13]:
result

,address,latitude,longitude,manageInstitution,managementNumber,ntatcSeq,parkArea,regDtTm,section,tel,title,dong
0,대전광역시 대덕구 장동 457 일원,36.406849,127.441681,대전광역시 공원관리사업소(공원관리과),63000-00016,1081734097,85488,2015-12-31,문화공원,042-270-7860,장동,대덕구
1,대전광역시 대덕구 연축동 산31 일원,36.388015,127.437176,대전광역시 공원관리사업소(공원관리과),63000-00015,1081734096,264600,2015-12-31,근린공원,042-270-7860,연축,대덕구
2,대전광역시 동구 효평동 산84-1 일원,36.393979,127.455059,대전광역시 공원관리사업소(공원관리과),63000-00014,1081734095,82032,2015-12-31,역사공원,042-270-7860,계족산성,동구
3,대전광역시 대덕구 장동 130-2 일원,36.403092,127.445508,대전광역시 공원관리사업소(공원관리과),63000-00013,1081734094,86800,2015-12-31,문화공원,042-270-7860,산디,대덕구
4,대전광역시 대덕구 읍내동 산1-9 일원,36.381933,127.437533,대전광역시 공원관리사업소(공원관리과),63000-00012,1081734093,347600,2015-12-31,문화공원,042-270-7860,읍내,대덕구
...,...,...,...,...,...,...,...,...,...,...,...,...
407,대전광역시 중구 호동 산101번지 일원,36.2975369,127.437172799999,대전광역시 공원관리사업소(공원관리과),30000-00005,1051456,407900,2016-06-10,근린공원,042-270-7860,호동근린공원,중구
408,대전광역시 중구 대사동 190-11번지,36.3094513,127.4239611,대전광역시 공원관리사업소(공원관리과),30000-00004,1051455,757000,2016-06-10,근린공원,042-270-7860,대사근린공원,중구
409,대전광역시 중구 사정동 산2번지,36.3007952,127.407088599999,대전광역시 공원관리사업소(공원관리과),30000-00003,1051454,1288000,2016-06-10,근린공원,042-270-7860,사정근린공원,중구
410,대전광역시 서구 둔산대로 169,36.215831,127.231643,대전광역시 한밭수목원,30000-00002,1051453,569340.6,2016-06-10,근린공원,042-270-8452,둔산대공원,서구


In [14]:
result.to_json('D:/python/daejeon_park.json')